In [2]:
#importing required libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.visualization import plot_histogram
simulator = Aer.get_backend('qasm_simulator')
import numpy as np

#Tranforms the state of the system, applying X gates according to 0's in the vector 'state'
def X_Transformation(qc, qreg, state):
    
    for j in np.arange(len(state)):
        if( int(state[j])==0 ):
            qc.x( qreg[int(j)] )

#picks out the marked state and applies a negative phase
def Grover_Oracle(mark, qc, q, an1):
    qc.h( an1[0] )
    X_Transformation(qc, q, mark)
    if( len(mark) > 2 ):
        qc.mct ( q , an1[0] , None , 'noancilla' )
        #mct( q , an1[0])
    if( len(mark) == 2 ):
        qc.ccx( q[0], q[1], an1[0] )
    X_Transformation(qc, q, mark)
    qc.h( an1[0] )
    

#ammends the instructions for a Grover Diffusion Operation to the program    
def Grover_Diffusion(mark, qc, q, an1):
    zeros_state = []
    for i in np.arange( len(mark) ):
        zeros_state.append( 0 )
        qc.h( q[int(i)] )
    Grover_Oracle(zeros_state, qc, q, an1)
    for j in np.arange( len(mark) ):
        qc.h( q[int(j)] )


#function for addition modulo 2^n
def adder(qc,a,b):
    for i in range(1,n):
        qc.cx(b[i],a[i])
    
    for i in reversed(range(1,n-1)):
        qc.cx(b[i],b[i+1])
    
    for i in range(n-1):
        qc.ccx(a[i],b[i],b[i+1])
    
    for i in reversed(range(1,n)):
        qc.cx(b[i],a[i])
        qc.ccx(a[i-1],b[i-1],b[i])
    
    for i in range(1,n-1):
        qc.cx(b[i],b[i+1])
    
    for i in range(n):
        qc.cx(b[i],a[i])

        
#function for inverse of addition modulo 2^n
def adderinv(qc,a,b):
    for i in reversed(range(n)):
        qc.cx(b[i],a[i])
        
    for i in reversed(range(1,n-1)):
        qc.cx(b[i],b[i+1])
    
    for i in range(1,n):
        qc.ccx(a[i-1],b[i-1],b[i])
        qc.cx(b[i],a[i])
     
    for i in reversed(range(n-1)):
        qc.ccx(a[i],b[i],b[i+1])
        
    for i in range(1,n-1):
        qc.cx(b[i],b[i+1])
        
    for i in reversed(range(1,n)):
        qc.cx(b[i],a[i])

        
#function for rotating a word left
def rotlt(qc,a):
    for i in reversed(range(n-1)):
        qc.cx(a[i+1],a[i])
        qc.cx(a[i],a[i+1])
        qc.cx(a[i+1],a[i])        

#function for rotating a word right
def rotrt(qc,a):
    for i in range(n-1):
        qc.cx(a[i+1],a[i])
        qc.cx(a[i],a[i+1])
        qc.cx(a[i+1],a[i])
     
    
#function for the quarter round
def qtrround(qc,a,b):
    adder(qc,a,b)
    for i in range(n):
        qc.cx(a[i],b[i])
    for i in range(2):
        rotlt(qc,b)
      
    
#function for the inverse of quarter round
def qtrroundinv(qc,a,b):
    for i in range(2):
        rotrt(qc,b)
    for i in reversed(range(n)):
        qc.cx(a[i],b[i])
    adderinv(qc,a,b)
    
    
#function for updating the state 
def stateupdate(qc):
    for i in range(n):
        qc.cx(key[i],keyp[i])
    qc.x(con[0])
    qc.x(iv0[1])
    qc.x(iv1[2])
    qtrround(qc,con,iv0)
    qtrround(qc,iv1,key)
    qtrround(qc,key,con)
    qtrround(qc,iv0,iv1)
    adder(qc,key,keyp)
    
    
#function for inverting the state update 
def stupdateinv(qc):
    adderinv(qc,key,keyp)
    qtrroundinv(qc,iv0,iv1)
    qtrroundinv(qc,key,con)
    qtrroundinv(qc,iv1,key)
    qtrroundinv(qc,con,iv0)
    qc.x(iv1[2])
    qc.x(iv0[1])
    qc.x(con[0])
    for i in range(n):
        qc.cx(key[i],keyp[i])
        
#preparing the quantum circuit
n = 3

con = QuantumRegister(n, name = 'con')
iv0 = QuantumRegister(n, name = 'iv0')
iv1 = QuantumRegister(n, name = 'iv1')
key = QuantumRegister(n, name = 'key')
keyp = QuantumRegister(n, name = 'keyp')

trg = QuantumRegister(1, name = 'trg')
cl = ClassicalRegister(n, name='cl')
qc = QuantumCircuit(con,iv0,iv1,key,keyp,trg,cl)

qc.h(key)

qc.x(trg[0])

for j in range(5):
    stateupdate(qc)
    Grover_Oracle([1,1,0,1,1,1,0,1,0,0,0,1], qc, [con[0],con[1],con[2],iv0[0],iv0[1],iv0[2],iv1[0],iv1[1],iv1[2],key[0],key[1],key[2]], trg)
    stupdateinv(qc)
    Grover_Diffusion([0,0,0], qc, key, trg)
    

for i in range(n):
    qc.measure(key[i],cl[i])
        
job = execute(qc, simulator, shots=1024)
result = job.result()
counts = result.get_counts(qc)
print(counts)
plot_histogram(counts) 